In [1]:
#read in data
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np
import pymc as pm
import arviz as az
import os 

In [51]:
import numpy as np
import pandas as pd
import json

# Define servo indices (as you've already done)
servoIndices = {
    "NeckTilt": 0,
    "NeckPan": 1,
    "LeftEye": 2,
    "RightEye": 3,
    "LeftPupil": 4,
    "RightPupil": 5,
    "LeftArmJoint1": 6,
    "LeftArmJoint2": 7,
    "LeftArmJoint3": 8,
    "LeftArmJoint4": 9,
    "LeftArmJoint5": 10,
    "LeftHand": 11,
    "RightArmJoint1": 12,
    "RightArmJoint2": 13,
    "RightArmJoint3": 14,
    "RightArmJoint4": 15,
    "RightArmJoint5": 16,
    "RightHand": 17,
    "Body": 18
}

def parse_array(values):
    return np.array(values)

def read_data_json(file):
    # Initialize an empty DataFrame with required columns
    df = pd.DataFrame(columns=['UniqueID', 'Trajectory', 'Current', 'GyroX', 'GyroY', 'GyroZ', 'AccelX', 'AccelY', 'AccelZ'])
    
    with open(file) as f:
        data = json.load(f)
    
    # Extract data for each trajectory
    all_rows = []
    for traj in data['Trajectories']:
        unique_id = traj['UniqueID']
        trajectory = parse_array(traj['Trajectory'])
        current = parse_array(traj['Current'])
        gyro = parse_array(traj['Gyro'])
        accel = parse_array(traj['Accel'])
        
        # For each point in the trajectory, create a single row with individual values
        for i in range(len(trajectory)):
            row = {
                'UniqueID': unique_id,
                'Trajectory': trajectory[i],
                'Current': current[i],
                'GyroX': gyro[i, 0],
                'GyroY': gyro[i, 1],
                'GyroZ': gyro[i, 2],
                'AccelX': accel[i, 0],
                'AccelY': accel[i, 1],
                'AccelZ': accel[i, 2]
            }
            all_rows.append(row)
    
    # Create a DataFrame from the collected rows
    df = pd.DataFrame(all_rows)
    return df

# Example usage
df = read_data_json("data/TrajectoriesTorso.json")


In [52]:
#standardise element of the trajectory from every row

df['Trajectory'].apply(lambda x: x / np.linalg.norm(x))


0         [0.6757853609016968, 0.7370984642440679]
1         [0.6757853609016968, 0.7370984642440679]
2         [0.6757853609016968, 0.7370984642440679]
3         [0.6759726769613781, 0.7369266856354629]
4         [0.6757853609016968, 0.7370984642440679]
                            ...                   
700862    [0.7081452737034171, 0.7060667612425277]
700863    [0.7069300658590185, 0.7072834523616142]
700864    [0.7060834164515227, 0.7081286669893724]
700865    [0.7048574421543911, 0.7093489876214453]
700866    [0.7036268711679379, 0.7105696490636354]
Name: Trajectory, Length: 700867, dtype: object

In [68]:
#standardize the data
std_df = df.copy()
for column in df.columns:
    if column != 'UniqueID':
        std_df[column] = df[column].apply(lambda x: x / np.linalg.norm(x) if np.linalg.norm(x) != 0 else 0)
        std_df[column] = std_df[column].round(3)

In [72]:
std_df

,UniqueID,Trajectory,Current,GyroX,GyroY,GyroZ,AccelX,AccelY,AccelZ
0,699873,"[0.6757853609016968, 0.7370984642440679]",0,-1.0,1.0,-1.0,0.0,0.0,0.0
1,699873,"[0.6757853609016968, 0.7370984642440679]",0,-1.0,1.0,0.0,1.0,-1.0,-1.0
2,699873,"[0.6757853609016968, 0.7370984642440679]","[-0.7071067811865475, -0.7071067811865475]",-1.0,1.0,1.0,1.0,-1.0,-1.0
3,699873,"[0.6759726769613781, 0.7369266856354629]","[-0.7071067811865475, -0.7071067811865475]",-1.0,1.0,1.0,1.0,-1.0,-1.0
4,699873,"[0.6757853609016968, 0.7370984642440679]","[-0.7071067811865475, -0.7071067811865475]",-1.0,-1.0,1.0,1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...
700862,57131,"[0.7081452737034171, 0.7060667612425277]","[-0.9977851578566089, -0.06651901052377393]",1.0,1.0,-1.0,1.0,-1.0,-1.0
700863,57131,"[0.7069300658590185, 0.7072834523616142]","[-0.9985422732775083, -0.05397525801500045]",1.0,1.0,-1.0,1.0,-1.0,-1.0
700864,57131,"[0.7060834164515227, 0.7081286669893724]","[-0.9828721869343219, -0.18428853505018536]",1.0,1.0,-1.0,1.0,-1.0,-1.0
700865,57131,"[0.7048574421543911, 0.7093489876214453]","[-0.9962405881956828, -0.08662961636484198]",1.0,1.0,-1.0,1.0,-1.0,-1.0


In [30]:
std_df.head()

,UniqueID,Trajectory,Current,GyroX,GyroY,GyroZ,AccelX,AccelY,AccelZ
0,699873,"[176.35, 192.35]","[0.0, 0.0]",-0.027395,-0.032445,-0.020137,0.049532,0.086510,2.871988
1,699873,"[176.35, 192.35]","[0.0, 0.0]",0.011748,-0.027824,0.006174,0.405297,-0.009269,-1.550526
2,699873,"[176.35, 192.35]","[-23.52, -23.52]",0.015763,-0.029364,0.045335,0.405297,-0.009269,-1.550526
3,699873,"[176.44, 192.35]","[-26.88, -26.88]",-0.091630,-0.023202,0.037992,0.405297,-0.009269,-1.550526
4,699873,"[176.35, 192.35]","[-26.88, -26.88]",-0.027395,-0.062230,0.074706,0.405297,-0.009269,-1.550526


In [26]:
# for every unique ID, check the minimum current it took for the servo to move
min_current = std_df.groupby('UniqueID')['Current'].min()
min_current = min_current.reset_index()
min_current.columns = ['UniqueID', 'MinCurrent']

min_current


ValueError: agg function failed [how->min,dtype->object]